<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# Hello Replicator!

The goal of this tutorial is to provide an introduction to the basic Omniverse Replicator functionalities.  In this notebook, we'll step through:
1. Creating a simple scene with a few predefined 3D assets
2. Applying randomizations
3. Writing the generated images to the disk for further processing

<span style="color:red;">TODO: add image that characterizes the notebook</span>

**[1.0 Hello World Replicator](#1.0-Hello-World-Replicator)<br>**
**[2.0 Basic Randomizations with Replicator](#2.0-Basic-Randomizations-with-Replicator)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Background Randomization](#2.1-Background-Randomization)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Change Textures](#2.2-Change-Textures)<br>
**[3.0 Advanced Randomization and Challenge](#3.0-Advanced-Randomization-and-Challenge)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[3.1 Importing multiple assets from designated folder at once](#3.1-Importing-multiple-assets-from-designated-folder-at-once)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2 Add variety of furniture and semantically label them](#3.2-Add-variety-of-furniture-and-semantically-label-them)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.3 Exercise: Choose a variety of 5 furniture with labels, randomize their rotations, placements and the camera](#3.3-Exercise:)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.4 Suggested Solution to 3.3](#3.4)<br>

---
# 1.0 Hello World Replicator

[Omniverse Replicator](https://docs.omniverse.nvidia.com/prod_extensions/prod_extensions/ext_replicator.html) is a highly extensible framework that enables physically accurate 3D synthetic data generation to accelerate training and performance of AI perception networks. You can run Replicator from within the Omniverse Code application, which can be run from a desktop RTX GPU workstation.  

For the purposes of this workshop, we'll use a cloud-based instance of Omniverse Code running in an RTX GPU instance on the DLI (Deep Learning Institute) platform. Running 

First, let’s create a new USD layer using `rep.new_layer()` in which we will place and randomize our scene assets.  Layers let us isolate the changes we make to the base stage.

We will create a camera in the scene at a desired position. Cameras give us an eye to see into the stage.

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():
    camera = rep.create.camera(position=(0, 500, 1000), look_at=(0,0,0))

Now, let’s create basic 3D shapes for our simple scene. Users can also bring any custom 3D objects they may have already created. The tutorial [Using  Replicator with a fully developed scene](https://docs.omniverse.nvidia.com/prod_extensions/prod_extensions/ext_replicator/apis_with_fully_developed_scene.html) demonstrates the process to bring user generated assets within the scene. Notice when creating the assets, we add semantic data, using the flag `semantics`. These semantic labels are added to the 3D shapes and used for generating annotation data to be used in ML model training.

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():
    camera = rep.create.camera(position=(0, 500, 1000), look_at=(0,0,0))

    # Create simple shapes to manipulate
    plane = rep.create.plane(semantics=[('class', 'plane')], position=(0,-100,0), scale=(100,1,100))
    torus = rep.create.torus(semantics=[('class', 'torus')] , position=(200, 0 , 100))
    sphere = rep.create.sphere(semantics=[('class', 'sphere')], position=(0, 0, 100))
    cube = rep.create.cube(semantics=[('class', 'cube')],  position=(-200, 0, 100))

We have created a camera, several 3D assets and a plane to constrain those assets. At this point, we are ready to define randomizers with specified distributions (uniform random in this case). In this tutorial, we randomize the position and scale of these assets within the scene.

Notice that these randomizations are triggered at every frame event. We will generate 10 frames with randomizations in this example.

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():
    camera = rep.create.camera(position=(0, 500, 1000), look_at=(0,0,0))

    # Create simple shapes to manipulate
    plane = rep.create.plane(semantics=[('class', 'plane')], position=(0,-100,0), scale=(100,1,100))
    torus = rep.create.torus(semantics=[('class', 'torus')] , position=(200, 0 , 100))
    sphere = rep.create.sphere(semantics=[('class', 'sphere')], position=(0, 0, 100))
    cube = rep.create.cube(semantics=[('class', 'cube')],  position=(-200, 0, 100))
    
    # Randomize position and scale on each frame
    with rep.trigger.on_frame(num_frames=10):
        with rep.create.group([torus, sphere, cube]):
            rep.modify.pose(
                position=rep.distribution.uniform((-300, 0, -300), (300, 0, 300)),
                scale=rep.distribution.uniform(0.1, 2))

Lastly, data generated for each frame can then written to the disk. For this purpose, Replicator provides a default `Writer` that can be used as demonstrated in this example. In addition to the default writer, users can also create their own custom writer. This is demonstrated in one of the [subsequent tutorials](https://docs.omniverse.nvidia.com/prod_extensions/prod_extensions/ext_replicator/custom_writer.html).

First we will create a `render_product` that attaches to our scene camera.  We will set an output resolution of 1024x1024 for our output images.

The default Writer is initialized and is attached to the renderer to produce the output of the images, semantic segmentation, and annotations with 2D bounding boxes. Note the user can specify a specific output directory for the writer to use.

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():

    camera = rep.create.camera(position=(0, 500, 1000), look_at=(0,0,0))

    plane = rep.create.plane(semantics=[('class', 'plane')], position=(0,-100,0), scale=(100,1,100))
    torus = rep.create.torus(semantics=[('class', 'torus')] , position=(200, 0 , 100))
    sphere = rep.create.sphere(semantics=[('class', 'sphere')], position=(0, 0, 100))
    cube = rep.create.cube(semantics=[('class', 'cube')],  position=(-200, 0, 100))
    
    with rep.trigger.on_frame(num_frames=10):
        with rep.create.group([torus, sphere, cube]):
            rep.modify.pose(
                position=rep.distribution.uniform((-300, 0, -300), (300, 0, 300)),
                scale=rep.distribution.uniform(0.1, 2))

render_product = rep.create.render_product(camera, (1024, 1024))
writer = rep.WriterRegistry.get("BasicWriter")
writer.initialize(run_id=1, output_dir="replicator_example", rgb=True, semantic_segmentation=True, bounding_box_2d_tight=True)
writer.attach([render_product])

Now, let's see that script in action. To connect, you'll need to know the IP address of the remote server.  

Execute the following cell to get the IP address and connect via the Omniverse Streaming client. If for some reason, you cannot connect you will find the way of running headlessly below. 

In [6]:
from requests import get
ip=get('https://api.ipify.org').text
print('This instance IP address is: {}'.format(ip))

This instance IP address is: 34.219.32.45


After connecting to the instance, go to the Script Editor and you type-in the script below, or use the included `Snippet` from the drop-down menu of the Script Editor.

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():

    camera = rep.create.camera(position=(0, 500, 1000), look_at=(0,0,0))

    plane = rep.create.plane(semantics=[('class', 'plane')], position=(0,-100,0), scale=(100,1,100))
    torus = rep.create.torus(semantics=[('class', 'torus')] , position=(200, 0 , 100))
    sphere = rep.create.sphere(semantics=[('class', 'sphere')], position=(0, 0, 100))
    cube = rep.create.cube(semantics=[('class', 'cube')],  position=(-200, 0, 100))
    
    with rep.trigger.on_frame(num_frames=10):
        with rep.create.group([torus, sphere, cube]):
            rep.modify.pose(
                position=rep.distribution.uniform((-300, 0, -300), (300, 0, 300)),
                scale=rep.distribution.uniform(0.1, 2))

render_product = rep.create.render_product(camera, (1024, 1024))
writer = rep.WriterRegistry.get("BasicWriter")
writer.initialize(run_id=1, output_dir="replicator_example", rgb=True, semantic_segmentation=True, bounding_box_2d_tight=True)
writer.attach([render_product])


To preview our randomizations, we can use `rep.orchestrator.preview()` in the Script Editor, or we can choose `Preview` from the `Replicator` drop-down at the top of our Omniverse Code toolbar.

To generate all ten frames, change `rep.orchestrator.preview()` to `rep.orchestrator.run()`, or select `Run` from the `Replicator` drop-down menu. 

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():

    camera = rep.create.camera(position=(0, 500, 1000), look_at=(0,0,0))

    # Create simple shapes to manipulate
    plane = rep.create.plane(semantics=[('class', 'plane')], position=(0,-100,0), scale=(100,1,100))
    torus = rep.create.torus(semantics=[('class', 'torus')] , position=(200, 0 , 100))
    sphere = rep.create.sphere(semantics=[('class', 'sphere')], position=(0, 0, 100))
    cube = rep.create.cube(semantics=[('class', 'cube')],  position=(-200, 0, 100))
    
    # Randomize position and scale on each frame
    with rep.trigger.on_frame(num_frames=10):
        with rep.create.group([torus, sphere, cube]):
            rep.modify.pose(
                position=rep.distribution.uniform((-300, 0, -300), (300, 0, 300)),
                scale=rep.distribution.uniform(0.1, 2))

# Initialize render product and attach writer
render_product = rep.create.render_product(camera, (1024, 1024))
writer = rep.WriterRegistry.get("BasicWriter")
writer.initialize(output_dir="replicator_example", rgb=True, semantic_segmentation=True, bounding_box_2d_tight=True)
writer.attach([render_product])
rep.orchestrator.run()

Below you can also see how you would run replicator headlessly. This may be needed if your are running at scale, or if for some reason you cannot connect to the client, or you want Omniverse to exit after completing.

```!cd / && ./code_startup.sh  --allow-root --no-window --/omni/replicator/script=/dli/task/Replicator_scripts/hello_world.py```

In [ ]:
!cd / && ./code_startup.sh  --allow-root --no-window --/omni/replicator/script=/dli/task/Replicator_scripts/hello_world.py

---
# 2.0 Basic Randomizations with Replicator

## 2.1 Background Randomization

The goal of this tutorial is to provide an introduction to the basic randomizations that can be done using Replicator. 

In this case, Omniverse Replicator is running in an `RTX GPU` instance on the `DLI` (Deep Learning Institute) platform. Omniverse is accesible through Omniverse Remote client or can run headlessly. For the best experience we recommend the client as shown in the Getting started notebook. 
To run the script follow set up instructions in Setting up the Script Editor.

Building on what we have learnt in 1.0 Hello Replicator which is the basic scene set-up before randomizing placements of the basic shapes in the scene. Here, we guide you on how you can start to randomize your environments. First start by loading in the files of the various environments that you want to use for this series. In this case, we are using the 4 environments found within the Omniverse content browser, but there are many more to choose from. You can even import in your own .hdr files if you're trying this after you've completed the course.

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():
    def dome_lights():
        lights = rep.create.light(
            light_type="Dome",
            rotation= (270,0,0),
            texture=rep.distribution.choice([
                'omniverse://localhost/NVIDIA/Assets/Skies/Cloudy/champagne_castle_1_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Clear/evening_road_01_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Cloudy/kloofendal_48d_partly_cloudy_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Clear/qwantani_4k.hdr'
                ])
            )
        return lights.node

    rep.randomizer.register(dome_lights)

  Then we go ahead and create the object that we want to see in our scene, and in this case, we're choosing the torus that we used in 1.0. So we create that, then we go ahead to create the surface on which the object will reside, followed by the camera for this scene 

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():
    def dome_lights():
        lights = rep.create.light(
            light_type="Dome",
            rotation= (270,0,0),
            texture=rep.distribution.choice([
                'omniverse://localhost/NVIDIA/Assets/Skies/Cloudy/champagne_castle_1_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Clear/evening_road_01_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Cloudy/kloofendal_48d_partly_cloudy_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Clear/qwantani_4k.hdr'
                ])
            )
        return lights.node

    rep.randomizer.register(dome_lights)    
    torus = rep.create.torus(semantics=[('class', 'torus')] , position=(0, -200 , 100))
    
    #create surface
    surface = rep.create.disk(scale=5, visible=False)
    
    #create camera for the scene 
    camera = rep.create.camera()
    render_product = rep.create.render_product(camera, resolution=(1024, 1024))

Lastly, data generated for each frame is then written to the disk. Just like in the first example, we will use Replicator's built-in `BasicWriter` for this.

In the following code, we use the BasicWriter which is initialized and is attached to the renderer to produce the output of the images and annotations with 2D bounding boxes. Note the user can specify a specific output directory for the writer to use.

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():
    def dome_lights():
        lights = rep.create.light(
            light_type="Dome",
            rotation= (270,0,0),
            texture=rep.distribution.choice([
                'omniverse://localhost/NVIDIA/Assets/Skies/Cloudy/champagne_castle_1_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Clear/evening_road_01_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Cloudy/kloofendal_48d_partly_cloudy_4k.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Clear/qwantani_4k.hdr'
                ])
            )
        return lights.node

    rep.randomizer.register(dome_lights)    
    torus = rep.create.torus(semantics=[('class', 'torus')] , position=(0, -200 , 100))
    
    #create surface
    surface = rep.create.disk(scale=5, visible=False)
    
    #create camera & render product for the scene 
    camera = rep.create.camera()
    render_product = rep.create.render_product(camera, resolution=(1024, 1024))

    with rep.trigger.on_frame(num_frames=10,interval=10):
        rep.randomizer.dome_lights()
        with rep.create.group([torus]):
            rep.modify.pose(
                position=rep.distribution.uniform((-100, -100, -100), (200, 200, 200)),
                scale=rep.distribution.uniform(0.1, 2))
        with camera:
            rep.modify.pose(position=rep.distribution.uniform((-500, 200, 1000), (500, 500, 1500)), look_at=surface)

    # Initialize and attach writer
    writer = rep.WriterRegistry.get("BasicWriter")
    
    writer.initialize( output_dir="replicator_example_02", rgb=True)
    writer.attach([render_product])

## 2.2 Change Textures

So you've learnt how to change the background environments of your scene for randomization! 

Now to the next level!

Let’s create basic 3D shapes for our next scene. Users can also bring any custom 3D objects they may have already created. The tutorial Using the Replicator with a fully developed scene demonstrates the process to bring user generated assets within the scene. Notice when creating the assets, we add semantics, using the flag semantics. These semantics labels are added to the 3D shapes and used while generating annotations for the data.

In [ ]:
import omni.replicator.core as rep

#create new objects to be used in the dataset 
with rep.new_layer():
    sphere = rep.create.sphere(semantics=[('class', 'sphere')], position=(0, 100, 100), count=5)
    cube = rep.create.cube(semantics=[('class', 'cube')],  position=(200, 200 , 100),count=5 )
    cone = rep.create.cone(semantics=[('class', 'cone')],  position=(200, 400 , 200),count=10 )
    cylinder = rep.create.cylinder(semantics=[('class', 'cylinder')],  position=(200, 100 , 200),count=5 )

After we have created our objects - note that users can create as many as they want - we are going to create our scene camera, and attach it to our render product. This is similar to what we have done above. By attaching it to the render product, we're ensuring that the randomized elements of our scenes, are captured by the camera and sent to be written by the Basic Writer. 

In [ ]:
# create new camera 
camera = rep.create.camera(position=(0, 0, 1000))

# create render product and attach to camera
render_product = rep.create.render_product(camera, (1024, 1024))
#  create plane if needed (but unused here)
plane = rep.create.plane(scale=10, visible=True)

The lesson for this section comes here, and we now want to not only create our various objects in the scene, but we also want to make the textures randomized. So here we've added 4 textures for Replicator to choose from when alternating between the textures used for the randomization. Users are once again free to add their own textures, and even more textures if they want a higher degree of randomization to occur for this part. 


In [ ]:
# function to get shapes that you've created above 
    def get_shapes():
        shapes = rep.get.prims(semantics=[('class', 'cube'), ('class', 'sphere'), ('class', 'cone'),('class', 'cylinder')])
        with shapes:
        #assign textures to the different objects 
            rep.randomizer.texture(textures=[
                    'omniverse://localhost/NVIDIA/Materials/vMaterials_2/Ground/textures/aggregate_exposed_diff.jpg',
                    'omniverse://localhost/NVIDIA/Materials/vMaterials_2/Ground/textures/gravel_track_ballast_diff.jpg',
                    'omniverse://localhost/NVIDIA/Materials/vMaterials_2/Ground/textures/gravel_track_ballast_multi_R_rough_G_ao.jpg',
                    'omniverse://localhost/NVIDIA/Materials/vMaterials_2/Ground/textures/rough_gravel_rough.jpg'
                    ])
        
        # modify pose and distribution 
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 50, -500), (500, 50, 500)),
                rotation=rep.distribution.uniform((0,-180, 0), (0, 180, 0)),
                scale=rep.distribution.normal(1, 0.5)
            )
        return shapes.node
    # register the get shapes function as a randomizer function 
    rep.randomizer.register(get_shapes)

Similar to the previous sections, here we run our randomization function, and proceed to write the data generated for each frame. For this purpose, Replicator provides a default Writer that can be used as demonstrated in this example. In addition to the default writer, users can also create their own custom writer.

In the following code, we use the basic Writer which is initialized and is attached to the renderer to produce the output of the images and annotations with 2D bounding boxes. Note the user can specify a specific output directory for the writer to use.

In [ ]:
    # Setup randomization. 100 variations here from 'num_frames'
    with rep.trigger.on_frame(num_frames=100):
        rep.randomizer.get_shapes()
        
    # Initialize and attach writer
    writer = rep.WriterRegistry.get("KittiWriter")
    # my own directory but use your relevant directory here 
    writer.initialize( output_dir="replicator_example_22", rgb=True)

    writer.attach([render_product])
    
    rep.orchestrator.run()

---
# 3.0 Advanced Randomization and Challenge [WIP]

# 3.1 Importing multiple assets from designated folder at once

The goal of this tutorial is to help users apply what they have learnt in the previous sections to do a more advanced application of Replicator.

Combining what we have learnt previously, now we seek to combine everything we have learnt and make a randomized dataset of furniture placements in various environments!

Here we show you how to load in assets using the localhost, and collect all the objects that are within a designated folder. 

In the first part of this code, we create our environments that we want to vary throughout our run of Replicator.

``` python

import omni.replicator.core as rep


with rep.new_layer():


    def dome_lights():

        lights = rep.create.light(
            light_type="Dome",
            rotation= (270,0,0),
            texture=rep.distribution.choice([
                'omniverse://localhost/NVIDIA/Assets/Skies/Indoor/ZetoCGcom_ExhibitionHall_Interior1.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Indoor/ZetoCG_com_WarehouseInterior2b.hdr',
                ])
            )
        return lights.node
    rep.randomizer.register(dome_lights)
```

Next, we will import assets from the Omniverse localhost library, more specifically, conference tables from the ArchVis library. Then, similar to what has been done previously, we will randomize the instance of the conference table chosen in this folder , then randomize the rotation and position of the folder. 

Note that here, the mode that is used is 'scene_instance' instead of the default, as this is needed while we work towards building a more complex scene.

``` python    
    conference_tables = 'omniverse://localhost/NVIDIA/Assets/ArchVis/Commercial/Conference/'


    # create randomizer function conference table assets. 
    #This randomization includes placement and rotation of the assets on the surface.
    def env_conference_table(size=5):
        confTable = rep.randomizer.instantiate(rep.utils.get_usd_files(conference_tables, recursive=False), size=size, mode='scene_instance')
        with confTable:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 0, -500), (500, 0, 500)),
                rotation=rep.distribution.uniform((-90,-180, 0), (-90, 180, 0)),
            )
        return confTable.node
    # Register randomization
    rep.randomizer.register(env_conference_table)
    
    # Setup camera and attach it to render product
    camera = rep.create.camera()
    render_product = rep.create.render_product(camera, resolution=(1024, 1024))
    

    

    surface = rep.create.disk(scale=100, visible=False)

    # trigger on frame for an interval
    with rep.trigger.on_frame(5):
        rep.randomizer.env_conference_table(2)        
        rep.randomizer.dome_lights()
        with camera:
            rep.modify.pose(position=rep.distribution.uniform((-500, 200, 1000), (500, 500, 1500)), look_at=surface)


    # Initialize and attach writer
    writer = rep.WriterRegistry.get("KittiWriter")
    writer.initialize(output_dir="furniture_rand_edit3_labelled", rgb=True)
    writer.attach([render_product])
```


## 3.2 Add variety of furniture and semantically label them

So in this section we are going to show you how to add various furnitures from different locations, and to label them semantically using Replicator's labelling capabilities. 

First, we are going to do something similiar to what we have seen earlier - to add new layer, and add the environments that we want to see changing in the scene. Then, we go ahead and import the objects from 5 different locations, pasting in their links and assigning them to their respective objects. Users can also go ahead and import objects from scenes that are pre-existing, as well as locations outside of Omniverse's localhost domain.

``` python

import omni.replicator.core as rep


with rep.new_layer():


    def dome_lights():

        lights = rep.create.light(
            light_type="Dome",
            rotation= (270,0,0),
            texture=rep.distribution.choice([
                'omniverse://localhost/NVIDIA/Assets/Skies/Indoor/ZetoCGcom_ExhibitionHall_Interior1.hdr',
                'omniverse://localhost/NVIDIA/Assets/Skies/Indoor/ZetoCG_com_WarehouseInterior2b.hdr',
                ])
            )
        return lights.node
    rep.randomizer.register(dome_lights)  
    
    # Add 5 different objects
    bar_stools = 'omniverse://localhost/NVIDIA/Assets/ArchVis/Residential/Furniture/BarStools/'
    reception_tables = 'omniverse://localhost/NVIDIA/Assets/ArchVis/Commercial/Reception/'
    chairs = 'omniverse://localhost/NVIDIA/Assets/ArchVis/Commercial/Seating/'
    normal_tables = 'omniverse://localhost/NVIDIA/Assets/ArchVis/Commercial/Tables/'
    sofa_set = 'omniverse://localhost/NVIDIA/Assets/ArchVis/Residential/Furniture/FurnitureSets/Dutchtown/'
```

In this next part, we are going to instantiate the folder that we read in for barstool objects, and load them in. 

Then, we go ahead to add semantic labels to this object, and in this case we are calling this object 'chair_barstool'. You are free to choose whichever label you would like to have for your specific folders and objects, and it will be the label that will follow all instances of such objects from the same folder, in your whole generated dataset.

```python

    # create randomizer function conference table assets. 
    #This randomization includes placement and rotation of the assets on the surface.
    def env_bar_stools(size=5):
        barstool = rep.randomizer.instantiate(rep.utils.get_usd_files(bar_stools, recursive=False), size=size, mode='scene_instance')
        rep.modify.semantics([('class', 'chair_barstool')])
        with barstool:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 0, -500), (500, 0, 500)),
                rotation=rep.distribution.uniform((-90,-180, 0), (-90, 180, 0)),
            )
        return barstool.node
    # Register randomization
    rep.randomizer.register(env_bar_stools)
    
  # Create functions for each object type / folder that you are importing assets from
  # function_2(.......)
  # function_3(.......)
  # function_4(.......)
  # function_5(.......)
    

```

## 3.3 Exercise: Choose a variety of 5 furniture with labels, randomize their rotations, placements and the camera
### Bonus Challenge: Add an additional randomized element e.g sphere_lights or vary the textures of the furniture in the scene

So now, for this exercise, it's your turn to create the script!

Here are the guiding steps for this exercise.

1) Define your function for your environment - how many environments HDRs would you like to change between?
2) Register this new function you've created - you'll need to call it later in the rep.trigger.on_frame() function
3) Decide on which are the 5 different furniture types you're planning to use, and what you will be labelling them as.
4) Copy their file paths, and import them into the script
5) Create randomizer functions for each of these 5 furniture types
6) Within these randomizer functions, you should initiate the path, then semantically label it, followed by modification of the poses of those objects - namely the placement and rotations.
(Bonus Challenge) If you would like to attempt the bonus challenge, create the sphere_lights function or a texture_randomizing function here
7) Register all these functions, so that you can call them later in the rep.trigger.on_frame() function
8) Create your viewing camera, as well as the surface that the camera will focus on
9) Attach your camera that you just created, to the render product, for writing data later on
10) Call the  with rep.trigger.on_frame(..) function, with all the relevant functions you've created, called within it
11) Write the data that is generated in each frame, to a specific folder, using either KittiWriter or BasicWriter.
12) Run the script, and view your very own generated synthetic data!

Refer to the previous sections to get hints on how to complete this exercise!

You can view the suggested solution in Section 3.4 below.

## 3.4 Solution for Exercise 3.3

### Congratulations! We believe you tried out the Exercise 3.3 and experienced great success in creating your own synthetic data! To give you more options and ideas on how such a script would look, below we have attached our suggested script on how we would complete Exercise 3.3!

### Happy experimenting!

In [ ]:
import omni.replicator.core as rep

with rep.new_layer():

    def dome_lights():
        lights = rep.create.light(
            light_type="Dome",
            rotation=(270, 0, 0),
            texture=rep.distribution.choice(
                [
                    "omniverse://localhost/NVIDIA/Assets/Skies/Indoor/ZetoCGcom_ExhibitionHall_Interior1.hdr",
                    "omniverse://localhost/NVIDIA/Assets/Skies/Indoor/ZetoCG_com_WarehouseInterior2b.hdr",
                ]
            ),
        )
        return lights.node

    rep.randomizer.register(dome_lights)

    bar_stools = (
        "omniverse://localhost/NVIDIA/Assets/ArchVis/Residential/Furniture/BarStools/"
    )
    reception_tables = (
        "omniverse://localhost/NVIDIA/Assets/ArchVis/Commercial/Reception/"
    )
    chairs = "omniverse://localhost/NVIDIA/Assets/ArchVis/Commercial/Seating/"
    normal_tables = "omniverse://localhost/NVIDIA/Assets/ArchVis/Commercial/Tables/"
    sofa_set = "omniverse://localhost/NVIDIA/Assets/ArchVis/Residential/Furniture/FurnitureSets/Dutchtown/"

    # create randomizer function conference table assets.
    # This randomization includes placement and rotation of the assets on the surface.
    def env_bar_stools(size=5):
        barstool = rep.randomizer.instantiate(
            rep.utils.get_usd_files(bar_stools, recursive=False),
            size=size,
            mode="scene_instance",
        )
        rep.modify.semantics([("class", "chair_barstool")])
        with barstool:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 0, -500), (500, 0, 500)),
                rotation=rep.distribution.uniform((-90, -180, 0), (-90, 180, 0)),
            )
        return barstool.node

    # Register randomization
    rep.randomizer.register(env_bar_stools)

    # create randomizer function conference table assets.
    # This randomization includes placement and rotation of the assets on the surface.
    def env_reception_table(size=5):
        receptTable = rep.randomizer.instantiate(
            rep.utils.get_usd_files(reception_tables, recursive=False),
            size=size,
            mode="scene_instance",
        )
        rep.modify.semantics([("class", "reception_table")])
        with receptTable:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 0, -500), (500, 0, 500)),
                rotation=rep.distribution.uniform((-90, -180, 0), (-90, 180, 0)),
            )
        return receptTable.node

    # Register randomization
    rep.randomizer.register(env_reception_table)

    # create randomizer function conference table assets.
    # This randomization includes placement and rotation of the assets on the surface.
    def env_chairs_notable(size=10):
        chairsEnv = rep.randomizer.instantiate(
            rep.utils.get_usd_files(chairs, recursive=False),
            size=size,
            mode="scene_instance",
        )
        rep.modify.semantics([("class", "office_chairs")])
        with chairsEnv:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 0, -500), (500, 0, 500)),
                rotation=rep.distribution.uniform((-90, -180, 0), (-90, 180, 0)),
            )
        return chairsEnv.node

    # Register randomization
    rep.randomizer.register(env_chairs_notable)

    # create randomizer function conference table assets.
    # This randomization includes placement and rotation of the assets on the surface.
    def env_normal_table(size=5):
        normTable = rep.randomizer.instantiate(
            rep.utils.get_usd_files(normal_tables, recursive=False),
            size=size,
            mode="scene_instance",
        )
        rep.modify.semantics([("class", "normal_table")])
        with normTable:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 0, -500), (500, 0, 500)),
                rotation=rep.distribution.uniform((-90, -180, 0), (-90, 180, 0)),
            )
        return normTable.node

    # Register randomization
    rep.randomizer.register(env_normal_table)

    # create randomizer function conference table assets.
    # This randomization includes placement and rotation of the assets on the surface.
    def env_sofaset(size=5):
        sofaset = rep.randomizer.instantiate(
            rep.utils.get_usd_files(sofa_set, recursive=False),
            size=size,
            mode="scene_instance",
        )
        rep.modify.semantics([("class", "sofa")])
        with sofaset:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 0, -500), (500, 0, 500)),
                rotation=rep.distribution.uniform((-90, -180, 0), (-90, 180, 0)),
            )
        return sofaset.node

    # Register randomization
    rep.randomizer.register(env_sofaset)

    # Setup camera and attach it to render product
    camera = rep.create.camera()
    render_product = rep.create.render_product(camera, resolution=(1024, 1024))

    # sphere lights for bonus challenge randomization
    def sphere_lights(num):
        lights = rep.create.light(
            light_type="Sphere",
            temperature=rep.distribution.normal(6500, 500),
            intensity=rep.distribution.normal(35000, 5000),
            position=rep.distribution.uniform((-300, -300, -300), (300, 300, 300)),
            scale=rep.distribution.uniform(50, 100),
            count=num,
        )
        return lights.node

    rep.randomizer.register(sphere_lights)

    # create the surface for the camera to focus on
    surface = rep.create.disk(scale=100, visible=False)

    # trigger on frame for an interval
    with rep.trigger.on_frame(5):
        rep.randomizer.env_bar_stools(2)
        rep.randomizer.env_reception_table(2)
        rep.randomizer.env_chairs_notable(2)
        rep.randomizer.env_normal_table(2)
        rep.randomizer.env_sofaset(2)

        # rep.randomizer.sphere_lights(10)
        rep.randomizer.dome_lights()
        with camera:
            rep.modify.pose(
                position=rep.distribution.uniform((-500, 200, 1000), (500, 500, 1500)),
                look_at=surface,
            )

    # Initialize and attach writer
    writer = rep.WriterRegistry.get("BasicWriter")
    writer.initialize(output_dir="replicator_example_33", rgb=True)
    writer.attach([render_product])


---
<h2 style="color:green;">Congratulations!</h2>

In this lab, you have learned about:
- What is Omniverse Replicator
- Creating simple shapes, cameras, lights, and 3D assets from a Nucleus server
- Using Replicator to make randomizations on objects
- Writing images and annotations to disk

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>